(00:astropy_fits)=
# About ``astropy.fits``

## Basic Information about FITS
* **FITS** = see [this official web](https://fits.gsfc.nasa.gov) and the [latest FITS standard document](https://fits.gsfc.nasa.gov/standard40/fits_standard40aa-le.pdf).
    * In section 4.4.2 of the document above, some standard keywords which you will most frequently see are described, including ``DATE-OBS``, ``OBJECT``, ``TELESCOP``, ``INSTRUME``, ``OBSERVER``, ``BSCALE``, ``BZERO``, ``BUNIT``,  etc. 
    * Summary described in Appendix C.
    * WCS is briefly explained in Ch 8.
* **HDU** = Header Data Unit
* **HDU List** = python ``list`` of the HDU objects
* **MEF** = Multi-Extension FITS
* A HDU is composed of Primary header (optionally with data) and other header & data. The simplest one contains only the primary HDU. In many cases, more than one such HDU is present in one FITS file. That is called the MEF.


## Prerequisites

Before you start this notebook, 

**MUST READ**:
* [Viewing and manipulating FITS images](https://learn.astropy.org/tutorials/FITS-images.html): Tutorial using Horsehead nebula image
* [Same as above](http://eso-python.github.io/ESOPythonTutorials/FITS-images.html) but slightly different and the outputs are a bit outdated.

**Highly recommended to read**:
* [learn astropy](https://learn.astropy.org/)
* About FITS image from an [astropy documentation](https://docs.astropy.org/en/stable/io/fits/usage/image.html).

**Other References**

If you want, **you may also look at these advaced tutorials** for your own research purposes:
* Visit [learn astropy](https://learn.astropy.org/) and select `fits` on the left sidebar

In [1]:
from pathlib import Path
import numpy as np
import time

from astropy.io import fits
from astropy.nddata import CCDData

## FITS from `Astropy`

### Making FITS  from ndarray
If you only have ndarray (e.g., numpy array), how can we convert it to FITS?

#### Making and Saving with ``CCDData``
The simplest way is to use ``CCDData``:

In [2]:
test_data_01 = np.ones((100, 100))
test_ccd_01 = CCDData(data=test_data_01, header=None, unit='adu')
print(type(test_ccd_01))
print(test_ccd_01.data)

<class 'astropy.nddata.ccddata.CCDData'>
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [3]:
test_ccd_01.header

OrderedDict()

Note that 
1. ``unit`` is a mandatory input for ``CCDData`` (which is, in my humble opinion, very annoying...)
2. The ``test_ccd_01.header`` does not contain anything and returned as ``OrderedDict()``. 

**It is always recommended to attach header to each FITS file**, but let me just save it for simple example:

In [4]:
test_ccd_01.write("test_01.fits", overwrite=True)

There is a specifically dedicated writer [``fits_ccddata_writer``](https://docs.astropy.org/en/stable/api/astropy.nddata.fits_ccddata_writer.html#astropy.nddata.fits_ccddata_writer) which is used to write a ``CCDData`` object to FITS file. Look at the doc for detailed saving options.

#### Making and Saving FITS data with ``astropy.fits``
The ``fits`` is more general than ``CCDData``, because, by principle, FITS should be able to contain virtually anything (including CCD image, star catalog, simulation data, etc). Thus, it's more complicated if you are only interested in the data from CCD (now you understand why ``CCDData`` is named so).

There are, thus, at least two ways to save our 2-D CCD-like data:

1. ``PrimaryHDU()``: A simple choice for the single extension case
2. ``ImageHDU()``: A choice for the MEF case

I here will show you how to make ``HDUList`` and save it using simple ``PrimaryHDU()``. Below there's a way for MEF in this note. Also you can check the [astropy doc on HDU](https://docs.astropy.org/en/stable/io/fits/api/hdus.html) for other classes of HDU.

In [5]:
test_data_01 = np.ones((100, 100))
test_hdu_01 = fits.PrimaryHDU(data=test_data_01, header=None)
print(type(test_hdu_01))
print(test_hdu_01.data)

<class 'astropy.io.fits.hdu.image.PrimaryHDU'>
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [6]:
# If you want, you can make HDUList.
# But it is unnecessary because we have only one extension.
# hdul = fits.HDUList([test_hdu_01])

test_hdu_01.writeto("test_01.fits", overwrite=True, output_verify='fix')

For the options when writing, please look at the official document ([here](https://astropy.readthedocs.io/en/stable/io/fits/api/files.html#astropy.io.fits.writeto)).

### Reading FITS

#### Reading with ``CCDData.read()``

Let's read this:

In [7]:
test_ccd_01_read = CCDData.read("test_01.fits", unit="adu")
print(type(test_ccd_01_read))
print(test_ccd_01_read.data)

<class 'astropy.nddata.ccddata.CCDData'>
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [8]:
test_ccd_01_read.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  100                                                  
NAXIS2  =                  100                                                  
EXTEND  =                    T                                                  

Note now that the header is automatically generated when it is being saved!

The **mandatory** keywords are generated (see the official standard FITS documentation in the beginning of this notebook).

This means, 
1. ``SIMPLE = T``: It follows the FITS standard
2. ``BITPIX = -64``: The data is in 64-bit float (see below)
3. ``NAXIS = 2``: The data has 2 axis.
4. ``NAXIS1 = 100`` and ``NAXIS2 = 100``: The data has 1st axis (X-axis, which is ``numpy axis 1``) of length 100 and the 2nd axis (Y-axis, which is ``numpy axis 0``) of length 100.
5. ``BUNIT = 'adu    '``: The unit of the pixel is in ADU.

From [astropy doc](https://docs.astropy.org/en/stable/io/fits/usage/image.html):

        BITPIX    Numpy Data Type
        8         numpy.uint8 (note it is UNsigned integer)
        16        numpy.int16
        32        numpy.int32
        -32       numpy.float32
        -64       numpy.float64

#### Reading with ``fits.open()``
Once it is saved, you can also read the FITS using ``astropy.fits``:

In [9]:
test_ccd_01_hdul = fits.open("test_01.fits")
print(type(test_ccd_01_hdul))

<class 'astropy.io.fits.hdu.hdulist.HDUList'>


Now it's not ``CCDData``, but ``HDUList`` object. It's like a python list, so you need to specify the index based on the information:

In [10]:
print(test_ccd_01_hdul.info())

Filename: test_01.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   (100, 100)   float64   
None


In [11]:
print(type(test_ccd_01_hdul[0]))
print(type(test_ccd_01_hdul["PRIMARY"]))

<class 'astropy.io.fits.hdu.image.PrimaryHDU'>
<class 'astropy.io.fits.hdu.image.PrimaryHDU'>


As can be seen, you can use index or ``Name`` in the ``.info()`` result.

In [12]:
print(test_ccd_01_hdul[0].data)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [13]:
test_ccd_01_hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  100                                                  
NAXIS2  =                  100                                                  
EXTEND  =                    T                                                  

Now you have identical result as the above ``CCDData.read`` case.

You can also use other methods (see [File Handling and Convenience Functions](https://docs.astropy.org/en/stable/io/fits/api/files.html))

* ``getheader()``: only gets header
* ``getdata()``: only gets data

But if you need both header and data, **DON'T** use these, but just use ``.open()``. These are useful only when you need either one of the two. 

* **NOTE**: It's also not recommended to use ``setval()``, ``delval()``, etc, unless you have definite reasons, because these overwrite the existing information. What if you have already used one of these but then found a mistake/bug in your code..? You have to re-download all the FITS files to start over again.

### Adding/Manipulating Header
There are several ways to add header using [``Header``](https://docs.astropy.org/en/stable/io/fits/api/headers.html).

#### ``Header.fromstring()``
The most primitive way. Likely this is used only if you want to make a fixed example (such as bug report) or tutorial like this notebook.

In [14]:
test_data_02 = np.ones((10, 20))

header_str = '''OBJECT  = 'dark    '                                                            
GAIN    =    1.360000014305115 / [e-/ADU] The electron gain factor.             
RDNOISE =                  9.0 / [e-] The (Gaussian) read noise.                
'''
header = fits.Header.fromstring(header_str, sep='\n')
test_ccd_02 = CCDData(data=test_data_02, header=header, unit='adu')

In [15]:
test_ccd_02.header

OBJECT  = 'dark    '                                                            
GAIN    =    1.360000014305115 / [e-/ADU] The electron gain factor.             
RDNOISE =                  9.0 / [e-] The (Gaussian) read noise.                

In [16]:
test_ccd_02.write("test2.fits", overwrite=True)
test_ccd_02_read = CCDData.read("test2.fits")

In [17]:
test_ccd_02_read.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   20                                                  
NAXIS2  =                   10                                                  
OBJECT  = 'dark    '                                                            
GAIN    =    1.360000014305115 / [e-/ADU] The electron gain factor.             
RDNOISE =                  9.0 / [e-] The (Gaussian) read noise.                
BUNIT   = 'adu     '                                                            

There are other similar methods, ``.fromtextfile()`` (when you have header in text file) and ``fromkeys()`` (when you have header in something like ``dict``). See above ``Header`` doc.

#### Setting 
There are many ways to set header keyword, value and comment, once a ``Header`` object is given.

1. Using ``header[<key>] = <value>``.
2. Using ``header[<key>] = (<value>, <comment>)``.
3. Using ``header.set(<key>, <value>, [<comment>])``.
4. Using ``header.add_comment(<comment_message>)``.
5. Using ``header.add_history(<history>)``.

The ``add_comment`` is to add the key ``COMMENT``. ``COMMENT`` is to put information such as reference, explanation, etc, which are not very important to be included as the key-value pair. 

The ``add_history`` is to add the key ``HISTORY``. ``HISTORY`` is to put information such as what is done at when.

Both ``COMMENT`` and ``HISTORY`` will appear at the end of the header, no matter at which time you added these.

In [18]:
header = fits.Header.fromstring('')  # empty header

# 1) Basic setting
header["object"] = 'dark'  # header key will automatically be capitalized

# 2) With comment
header["GAIN"] = (1.36, "[e-/ADU] The electron gain factor.")

# 3) Using .set()
header.set("RDNOISE", 9.0, "[e-] The (Gaussian) read noise.")
header.set("RDNOISE", 10.0, "[e-] The (Gaussian) read noise. Oops, I am adding it again! What will happen?")

# 4) Adding COMMENT line
header.add_comment("This is a testing fits file.")
header.add_comment("This is the second comment.")
header.add_comment("What if the comment is too long? "*6)

# 5) Adding HISTORY line
header.add_history("Bias subtracted 2019-01-01T00:00:01")
header.add_history("Dark corrected 2019-01-01T00:00:02")
header.add_history("Flat corrected 2019-01-01T00:00:03")
header.add_history("Cosmic ray rejected 2019-01-01T00:00:04")
header.add_history("WCS added 2019-01-01T00:00:06")

test_ccd_03 = CCDData(data=test_data_02, header=header, unit='adu')

In [19]:
test_ccd_03.header

OBJECT  = 'dark    '                                                            
GAIN    =                 1.36 / [e-/ADU] The electron gain factor.             
RDNOISE =                 10.0 / [e-] The (Gaussian) read noise. Oops, I am addi
COMMENT This is a testing fits file.                                            
COMMENT This is the second comment.                                             
COMMENT What if the comment is too long? What if the comment is too long? What i
COMMENT f the comment is too long? What if the comment is too long? What if the 
COMMENT comment is too long? What if the comment is too long?                   
HISTORY Bias subtracted 2019-01-01T00:00:01                                     
HISTORY Dark corrected 2019-01-01T00:00:02                                      
HISTORY Flat corrected 2019-01-01T00:00:03                                      
HISTORY Cosmic ray rejected 2019-01-01T00:00:04                                 
HISTORY WCS added 2019-01-01

From the result in ``RDNOISE``, you can see the header is *overwritten* by the newer value and comment. Also, if the comment is too long, the comment will be truncated as the ``WARNING`` message says.

From the long comment in ``COMMENT``, you can see the linebreak occurs when it's too long.

In [20]:
test_ccd_03.write("test3.fits", overwrite=True)
test_ccd_03_read = CCDData.read("test3.fits")

In [21]:
test_ccd_03_read.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   20                                                  
NAXIS2  =                   10                                                  
OBJECT  = 'dark    '                                                            
GAIN    =                 1.36 / [e-/ADU] The electron gain factor.             
RDNOISE =                 10.0 / [e-] The (Gaussian) read noise. Oops, I am addi
BUNIT   = 'adu     '                                                            
COMMENT This is a testing fits file.                                            
COMMENT This is the second comment.                                             
COMMENT What if the comment is too long? What if the comment is too long? What i
COMMENT f the comment is too

### Making/Reading MEF (optional)
If you are not advanced FITS user, it's generally better **NOT** to think about **making** MEF. It's better to stick to single-extension FITS format. 

But for many cases, e.g., HST images, may contain MEF, because there are clear reasons. Thus, I add how to deal with MEF here.

In [22]:
prim = fits.PrimaryHDU(data=None, header=header)
im1 = fits.ImageHDU(data=np.ones((10, 10)), header=None, name="SCI")
im2 = fits.ImageHDU(data=np.zeros((10, 10)), header=None, name="UNCERT")
hdul_mef = fits.HDUList([prim, im1, im2])
hdul_mef.writeto("test_mef.fits", overwrite=True, output_verify='fix')

In [23]:
hdul_mef_read = fits.open("test_mef.fits")
print(hdul_mef_read.info())

Filename: test_mef.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      17   ()      
  1  SCI           1 ImageHDU         8   (10, 10)   float64   
  2  UNCERT        1 ImageHDU         8   (10, 10)   float64   
None


Each extension can be accessed by either (1) index (``No.``) or (2) ``Name``.

In [24]:
np.testing.assert_allclose(hdul_mef_read[1].data, hdul_mef_read["SCI"].data)

The ``getdata()`` or ``getheader()``, etc, has ``ext`` and ``extname`` to select only one extension:

In [25]:
hdul_mef_data_1 = fits.getdata("test_mef.fits", ext=1)
hdul_mef_data_2 = fits.getdata("test_mef.fits", extname="UNCERT")
np.testing.assert_allclose(hdul_mef_data_1, hdul_mef_data_1 - hdul_mef_data_2)

Note that, ``CCDData`` understands any extension named ``'UNCERT'`` as the standard deviation uncertainty map (or simple 1-sigma error map), and reads it as ``astropy.nddata.nduncertainty.StdDevUncertainty``. Similarly, ``'MASK'`` is understood as mask. This is a reason why HST uses MEF than single-extension.

You can see this in our example:

In [26]:
test_ccd_mef_read = CCDData.read("test_mef.fits", unit='adu')

INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]


In [27]:
test_ccd_mef_read.data

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [28]:
print(type(test_ccd_mef_read.uncertainty))
test_ccd_mef_read.uncertainty

<class 'astropy.nddata.nduncertainty.StdDevUncertainty'>


StdDevUncertainty([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [29]:
tmpfitsfiles = Path('.').glob("test*.fits")
for fpath in tmpfitsfiles:
    fpath.unlink()